In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from kuma_utils.utils import sigmoid
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import Ridge

from datasets import *
from transforms import *
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [52]:
test = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/sample_submission.csv')
valid = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v23v.csv')
public748 = pd.read_csv('input/public748.csv')
iafoss_v5160 = {
    'oof': pd.read_csv('input/predictions/val_model_v5_1_6_0.csv'),
    'pred': pd.read_csv('input/predictions/submission_model_v5_1_6_0.csv')}
iafoss_v5210 = {
    'oof': pd.read_csv('input/predictions/val_model_v5_2_1_0.csv'),
    'pred': pd.read_csv('input/predictions/submission_model_v5_2_1_0.csv')}
iafoss_v7000 = {
    'oof': pd.read_csv('input/predictions/val_model_v7_0_0_0.csv'),
    'pred': pd.read_csv('input/predictions/submission_model_v7_0_0_0.csv')}
iafoss_v8000 = {
    'oof': pd.read_csv('input/predictions/val_model_v8_0_0_0c.csv'),
    'pred': pd.read_csv('input/predictions/submission_model_v8_0_0_0c.csv')}
chris_v16 = {
    'oof': pd.read_csv('input/predictions/valset_16_0.7036.csv'),
    'pred': pd.read_csv('input/predictions/sub_16_bce_mse.csv')}
chris_v18 = {
    'oof': pd.read_csv('input/predictions/valset_18_0.6997.csv'),
    'pred': pd.read_csv('input/predictions/sub_18_double_model.csv')}
drhb_exp200 = {
    'oof': pd.read_csv('input/predictions/EXP_200_BASELINE_CASHE_V4_EVAL.csv', usecols=['id', 'pred']),
    'pred': pd.read_csv('input/predictions/EXP_200_BASELINE_CASHE_V4.csv')}

In [88]:
model_paths = [
    # Path('results/ds_09_val0'),
    # Path('results/ds_09_val1'),
    # Path('results/aug_04'),
    # # Path('results/ds_14'),
    # # Path('results/ds_15'),
    # # Path('results/ds_16'),
    # # Path('results/model_02'),
    # Path('results/ds_14_prep2'),
    Path('results/ds_17_prep2'),
    Path('results/ds_18_prep3'),
    # Path('results/ds_18_prep6'),
    Path('results/ds_18_prep7'),
]


In [89]:
prediction_df = {}
oof_df = {}
for p in model_paths:
    pred = sigmoid(np.load(p/'predictions_tta.npy'))
    oof = sigmoid(np.load(p/'outoffolds_tta.npy'))

    if pred.shape[1] == 1:
        pred = pred.reshape(-1)
    elif pred.shape[0] == 5:
        pred = pred[0].reshape(-1)
    prediction_df[p.stem] = pred
    oof_df[p.stem] = oof.reshape(-1)
prediction_df = pd.DataFrame(prediction_df)
prediction_df['id'] = test['id']
# prediction_df['iafoss_v5160'] = iafoss_v5160['pred'].merge(prediction_df[['id']], on='id', how='right')['target']
# prediction_df['iafoss_v5210'] = iafoss_v5210['pred'].merge(prediction_df[['id']], on='id', how='right')['target']
# prediction_df['iafoss_v7000'] = iafoss_v7000['pred'].merge(prediction_df[['id']], on='id', how='right')['target']
prediction_df['iafoss_v8000'] = iafoss_v7000['pred'].merge(prediction_df[['id']], on='id', how='right')['target']
# prediction_df['drhb_exp200'] = drhb_exp200['pred'].merge(prediction_df[['id']], on='id', how='right')['target']
# prediction_df['chris_v16'] = chris_v16['pred'].merge(prediction_df[['id']], on='id', how='right')['target']
# prediction_df['chris_v18'] = chris_v18['pred'].merge(prediction_df[['id']], on='id', how='right')['target']
oof_df = pd.DataFrame(oof_df)
oof_df['id'] = valid['id']
# oof_df['iafoss_v5160'] = iafoss_v5160['oof']['target']
# oof_df['iafoss_v5210'] = iafoss_v5210['oof']['target']
# oof_df['iafoss_v7000'] = iafoss_v7000['oof']['target']
# oof_df['drhb_exp200'] = drhb_exp200['oof']['pred']
# oof_df['chris_v16'] = chris_v16['oof']['target']
# oof_df['chris_v18'] = chris_v18['oof']['target']
oof_df['iafoss_v8000'] = iafoss_v8000['oof']['target']

In [90]:
oof_df['target'] = valid['target']
oof_df.dropna(inplace=True)
oof_df

,ds_17_prep2,ds_18_prep3,ds_18_prep7,id,iafoss_v8000,target
0,0.502006,0.444413,0.462935,00054c878_gen,0.484375,0
1,0.453903,0.499001,0.498718,0007285a3_gen,0.525391,1
2,0.501457,0.521849,0.556014,00076c5a6_gen,0.560059,0
3,0.999999,1.000000,0.999980,001349290_gen,0.987793,1
4,0.689659,0.470522,0.470214,001a52e92_gen,0.454834,0
...,...,...,...,...,...,...
7970,0.999994,0.999990,0.999991,ffbce04ef_gen,0.980957,1
7971,0.458032,0.517230,0.547709,ffc2d976b_gen,0.491455,0
7972,0.411013,0.417791,0.418936,ffc905909_gen,0.454346,1
7973,0.923351,0.997500,0.972312,ffe276f3e_gen,0.499023,1


In [91]:
prediction_df

,ds_17_prep2,ds_18_prep3,ds_18_prep7,id,iafoss_v8000
0,0.461123,0.447986,0.439505,00054c878,0.513184
1,0.915820,0.995961,0.999387,0007285a3,0.991699
2,0.426183,0.409883,0.453477,00076c5a6,0.432129
3,0.747296,0.787974,0.826588,001349290,0.781738
4,0.920325,0.897227,0.913130,001a52e92,0.987793
...,...,...,...,...,...
7970,0.999937,0.999979,0.999998,ffbce04ef,0.999512
7971,0.477774,0.552246,0.591674,ffc2d976b,0.608887
7972,0.422349,0.439899,0.484202,ffc905909,0.475098
7973,0.396737,0.437145,0.430659,ffe276f3e,0.550293


In [92]:
model = Ridge(positive=True)
X = oof_df.drop(['target', 'id'], axis=1)
y = oof_df['target']
X_test = prediction_df.drop('id', axis=1)
model.fit(X, y)
y_pred = model.predict(X)
y_stack = model.predict(X_test)
print('stacking cv', roc_auc_score(y, y_pred))
print('weight', model.coef_)

stacking cv 0.7845627388606178
weight [0.         0.41864351 0.38048044 0.2553635 ]


In [87]:
y_stack

array([0.45147966, 1.01502667, 0.41663793, ..., 0.45279809, 0.45579608,
       0.44741343])

In [74]:
# test['target'] = prediction_df['aug_04'] * 0.2 + prediction_df['ds_05_aug2'] * 0.2 + prediction_df['public748'] * 0.6
# test['target'] = prediction_df['ds_09'] * 0.33 + prediction_df['ds_13'] * 0.33 + prediction_df['public748'] * 0.33
# test['target'] = prediction_df['ds_18_prep7']
test['target'] = y_stack

In [75]:
test

,id,target
0,00054c878,0.449435
1,0007285a3,1.014061
2,00076c5a6,0.418101
3,001349290,0.807625
4,001a52e92,0.938910
...,...,...
7970,ffbce04ef,1.017971
7971,ffc2d976b,0.575419
7972,ffc905909,0.453330
7973,ffe276f3e,0.451007


In [76]:
test.to_csv('results/stacking_submission_ds17prep2_ds18prep3_ds18prep7_iafoss8000c.csv', index=False)
# test.to_csv(Path('results/ds_18_prep7')/'submission.csv', index=False)

# Trad blending

In [15]:
blend_df = dict(
    # COMB7=pd.read_csv('input/predictions/COMB_7.csv')['target'], # LB 0.771
    # stacking_res=y_stack, # stack with only my model cv 0.739
    # drhb_exp200=prediction_df['drhb_exp200'], # LB 0.754 (add because it has no CV/LB corr)
    # iafoss_5160=prediction_df['iafoss_v5160'], # LB 0.758
    # iafoss_5210=prediction_df['iafoss_v5210'], # LB 0.757
    # public761=pd.read_csv('input/public761.csv')['target']
    COMB13_RB=pd.read_csv('input/predictions/COMB_13_REAL_BLENDING_0.4_0.6.csv').merge(test['id'], on='id', how='right')['target'],
    # ds_17_prep2=pd.read_csv('results/ds_17_prep2/submission.csv').merge(test['id'], on='id', how='right')['target'],
    # ds_17_prep1=pd.read_csv('results/ds_17_prep1_beforefix/submission.csv').merge(test['id'], on='id', how='right')['target'],
    ds_18_prep3=pd.read_csv('results/ds_18_prep3/submission.csv').merge(test['id'], on='id', how='right')['target'],
    ds_18_prep7=pd.read_csv('results/ds_18_prep7/submission.csv').merge(test['id'], on='id', how='right')['target'],
)
blend_df = pd.DataFrame(blend_df)
blend_df = blend_df.rank(axis=0)
blend_df = blend_df / blend_df.max(axis=0).values[None, :]

In [16]:
blend_df.corr()

,COMB13_RB,ds_18_prep3,ds_18_prep7
COMB13_RB,1.000000,0.843587,0.839268
ds_18_prep3,0.843587,1.000000,0.954660
ds_18_prep7,0.839268,0.954660,1.000000


In [28]:
blend_df['target'] = \
    blend_df['COMB13_RB'] * 0.8 + blend_df['ds_17_prep2'] * 0.1 + blend_df['ds_17_prep1'] * 0.1

In [29]:
test['target'] = blend_df['target']
test.to_csv(Path('results/blend_comb13rb_ds17prep2_ds17prep1.csv'), index=False)